#Please change the runtime to GPU T4 to run this notebook

# Training

In [ ]:
#@title Install and import the necessary libraries
!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops datasets requests

In [ ]:
#@title Importing transformer, Trainer and other necessary libraries
import os
import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)

from trl import SFTTrainer

In [ ]:
#@title Defining base model, dataset and tokenizer

# Model
base_model = "microsoft/Phi-3-mini-128k-instruct"
new_model = "phi-3-clinical"

# Dataset
dataset = load_dataset("hackint0sh/small-data", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [ ]:
#@title Defining Quantization configuration, loading base model and setting trainer arguments

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = True,
    bf16 = False,
    per_device_train_batch_size = 6,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

In [ ]:
#@title Setting Trainer with some hyperparameter

peft_config = LoraConfig(
    r=64,                  # Rank of the low-rank matrices
    lora_alpha= 16,        # Scaling factor for the low-rank matrices
    lora_dropout=0.05,     # Dropout applied to LoRA modifications
    bias="none",           # Bias configuration in LoRA
    task_type="CAUSAL_LM", # Specifies the task type
    target_modules=["qkv_proj", "o_proj"]  # Corrected target modules based on the actual model structure
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="data",  # Adjust this field name based on your JSON structure
    max_seq_length=1200,
    tokenizer=tokenizer,
    args=training_arguments,
)


In [ ]:
#@title Training model

trainer.train()

In [ ]:
#@title Testing the model
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_response(prompt, min_length=50, max_length=200):
    # Generate text with parameters set to ensure completeness and control over the generation process.
    generated_results = generator(
        prompt,
        max_length=max_length,
        min_length=min_length,
        num_return_sequences=1,
        do_sample=False,
        temperature=0.9,
        eos_token_id=tokenizer.eos_token_id
    )
    return generated_results[0]['generated_text']

# Prompt the model
prompt = None #@param
response = generate_response(prompt)
print(response)
